In [ ]:
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from community import community_louvain
from node2vec import Node2Vec
import pandas as pd


G = nx.karate_club_graph()


node_features = np.random.rand(G.number_of_nodes(), 16)

scaler = StandardScaler()
node_features = scaler.fit_transform(node_features)

data = from_networkx(G)
data.x = torch.tensor(node_features, dtype=torch.float)


class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

def compute_node_embeddings(data, num_clusters):
    model = GCN(data.x.shape[1], num_clusters)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    model.train()

    with torch.no_grad():
        initial_embeddings = model(data.x, data.edge_index)
        kmeans = KMeans(n_clusters=num_clusters).fit(initial_embeddings.detach().numpy())
        labels = torch.tensor(kmeans.labels_, dtype=torch.long)

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        embeddings = model(data.x, data.edge_index).numpy()
    return embeddings

def conductance(G, clusters):
    total_cut = 0
    for cluster in clusters:
        cut = nx.algorithms.cuts.cut_size(G, cluster)
        volume = sum(dict(G.degree(cluster)).values())
        if volume == 0:
            continue
        total_cut += cut / volume
    return total_cut / len(clusters)

num_clusters = 2


gcn_embeddings = compute_node_embeddings(data, num_clusters)


kmeans = KMeans(n_clusters=num_clusters).fit(gcn_embeddings)
hierarchical = AgglomerativeClustering(n_clusters=num_clusters).fit(gcn_embeddings)
spectral = SpectralClustering(n_clusters=num_clusters, affinity='nearest_neighbors').fit(gcn_embeddings)
louvain = community_louvain.best_partition(G)


def deepwalk_embedding(G, dimensions=64, walk_length=30, num_walks=200, workers=4):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)
    model = node2vec.fit(window=10, min_count=1)
    embeddings = np.array([model.wv[str(node)] for node in G.nodes()])
    return embeddings

deepwalk_embeddings = deepwalk_embedding(G)


node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4).fit()
node2vec_embeddings = np.array([node2vec.wv[str(node)] for node in G.nodes()])


deepwalk_clusters = KMeans(n_clusters=num_clusters).fit_predict(deepwalk_embeddings)
node2vec_clusters = KMeans(n_clusters=num_clusters).fit_predict(node2vec_embeddings)


ground_truth = np.array([0 if G.nodes[i]['club'] == 'Mr. Hi' else 1 for i in G.nodes()])


def compute_metrics(G, labels, ground_truth):
    modularity = nx.algorithms.community.quality.modularity(G, [list(np.where(labels == c)[0]) for c in np.unique(labels)])
    nmi = NMI(ground_truth, labels)
    ari = ARI(ground_truth, labels)
    conductance_value = conductance(G, [list(np.where(labels == c)[0]) for c in np.unique(labels)])
    return modularity, nmi, conductance_value, ari

metrics = {
    'KMeans': compute_metrics(G, kmeans.labels_, ground_truth),
    'Hierarchical Clustering': compute_metrics(G, hierarchical.labels_, ground_truth),
    'Spectral Clustering': compute_metrics(G, spectral.labels_, ground_truth),
    'Louvain Method': compute_metrics(G, np.array(list(louvain.values())), ground_truth),
    'DeepWalk': compute_metrics(G, deepwalk_clusters, ground_truth),
    'Node2Vec': compute_metrics(G, node2vec_clusters, ground_truth),
    'GCN': compute_metrics(G, kmeans.labels_, ground_truth)
}


df = pd.DataFrame(metrics, index=['Modularity', 'NMI', 'Conductance', 'ARI']).transpose()


print(df)


C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Computing transition probabilities:   0%|          | 0/34 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/34 [00:00<?, ?it/s]

                         Modularity       NMI  Conductance       ARI
KMeans                     0.085034  0.131983     0.213699  0.021601
Hierarchical Clustering    0.085034  0.131983     0.213699  0.021601
Spectral Clustering        0.157971  0.096232     0.359211  0.098088
Louvain Method             0.427672  0.582342     0.352778  0.446199
DeepWalk                   0.403628  0.837169     0.128289  0.882258
Node2Vec                   0.403628  0.837169     0.128289  0.882258
GCN                        0.085034  0.131983     0.213699  0.021601


C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AI-BIO\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
